In [2]:
import xarray as xr
import s3fs
import fsspec
import pandas as pd

# compatible library versions = python=3.10 xarray=2023.12.0 zarr=2.15.0 fsspec=2023.6.0 s3fs=2023.6.0 netCDF4 dask ipykernel jupyter -c conda-forge
# conda create -n aorc python=3.10 xarray=2023.12.0 zarr=2.15.0 fsspec=2023.6.0 s3fs=2023.6.0 netCDF4 dask ipykernel jupyter -c conda-forge


extra information:
Link to the directory to download - https://noaa-nws-aorc-v1-1-1km.s3.amazonaws.com/index.html

In [28]:
import fsspec

# Conectar no bucket
fs = fsspec.filesystem("s3", anon=True)

# Listar os diretórios dentro do bucket AORC
years = fs.ls("noaa-nws-aorc-v1-1-1km/")
print("Diretórios disponíveis no bucket:")
for y in years:
    print(y)


Diretórios disponíveis no bucket:
noaa-nws-aorc-v1-1-1km/1979.zarr
noaa-nws-aorc-v1-1-1km/1980.zarr
noaa-nws-aorc-v1-1-1km/1981.zarr
noaa-nws-aorc-v1-1-1km/1982.zarr
noaa-nws-aorc-v1-1-1km/1983.zarr
noaa-nws-aorc-v1-1-1km/1984.zarr
noaa-nws-aorc-v1-1-1km/1985.zarr
noaa-nws-aorc-v1-1-1km/1986.zarr
noaa-nws-aorc-v1-1-1km/1987.zarr
noaa-nws-aorc-v1-1-1km/1988.zarr
noaa-nws-aorc-v1-1-1km/1989.zarr
noaa-nws-aorc-v1-1-1km/1990.zarr
noaa-nws-aorc-v1-1-1km/1991.zarr
noaa-nws-aorc-v1-1-1km/1992.zarr
noaa-nws-aorc-v1-1-1km/1993.zarr
noaa-nws-aorc-v1-1-1km/1994.zarr
noaa-nws-aorc-v1-1-1km/1995.zarr
noaa-nws-aorc-v1-1-1km/1996.zarr
noaa-nws-aorc-v1-1-1km/1997.zarr
noaa-nws-aorc-v1-1-1km/1998.zarr
noaa-nws-aorc-v1-1-1km/1999.zarr
noaa-nws-aorc-v1-1-1km/2000.zarr
noaa-nws-aorc-v1-1-1km/2001.zarr
noaa-nws-aorc-v1-1-1km/2002.zarr
noaa-nws-aorc-v1-1-1km/2003.zarr
noaa-nws-aorc-v1-1-1km/2004.zarr
noaa-nws-aorc-v1-1-1km/2005.zarr
noaa-nws-aorc-v1-1-1km/2006.zarr
noaa-nws-aorc-v1-1-1km/2007.zarr
noaa-nws-

### Downloading a single timestep

In [46]:
target_time = "2024-12-31T00:00:00"
output_filename = "aorc_hourly_20241231_00zccc.nc"

In [47]:
# Define your bounding box
lat_min = 36.0
lat_max = 40.0
lon_min = -78.0
lon_max = -74.0

In [48]:
fs = fsspec.filesystem("s3", anon=True)
mapper = fs.get_mapper("noaa-nws-aorc-v1-1-1km/2024.zarr")

In [49]:
ds = xr.open_zarr(mapper, consolidated=True)
ds_hour = ds.sel(time=target_time) ## This downloads the entire CONUS


In [50]:
ds_hour = ds.sel(
     time=target_time,
     latitude=slice(lat_min, lat_max),
     longitude=slice(lon_min, lon_max)
 )

In [51]:
print(ds_hour)


<xarray.Dataset>
Dimensions:              (latitude: 480, longitude: 480)
Coordinates:
  * latitude             (latitude) float64 36.01 36.02 36.02 ... 39.99 40.0
  * longitude            (longitude) float64 -77.99 -77.99 ... -74.01 -74.0
    time                 datetime64[ns] 2024-12-31
Data variables:
    APCP_surface         (latitude, longitude) float32 dask.array<chunksize=(127, 159), meta=np.ndarray>
    DLWRF_surface        (latitude, longitude) float32 dask.array<chunksize=(127, 159), meta=np.ndarray>
    DSWRF_surface        (latitude, longitude) float32 dask.array<chunksize=(127, 159), meta=np.ndarray>
    PRES_surface         (latitude, longitude) float32 dask.array<chunksize=(127, 159), meta=np.ndarray>
    SPFH_2maboveground   (latitude, longitude) float32 dask.array<chunksize=(127, 159), meta=np.ndarray>
    TMP_2maboveground    (latitude, longitude) float32 dask.array<chunksize=(127, 159), meta=np.ndarray>
    UGRD_10maboveground  (latitude, longitude) float32 dask.arr

In [52]:
ds_hour.to_netcdf(rf"C:\Github\Lab_work\Pocomoke\validation_step3\{output_filename}", mode="w", format="NETCDF4")

print(f"✅ Saved: {output_filename}")

✅ Saved: aorc_hourly_20241231_00zccc.nc


## Downloading a time series

In [3]:
# Date range for August to October 2024
start_time = "2024-08-01T00:00:00"
end_time = "2024-10-30T23:59:00"
times = pd.date_range(start=start_time, end=end_time, freq="1H")

C:\Users\dveronez\AppData\Local\Temp\ipykernel_12928\142243935.py:4: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  times = pd.date_range(start=start_time, end=end_time, freq="1H")


In [4]:
# Define your bounding box
lat_min = 36.0
lat_max = 40.0
lon_min = -78.0
lon_max = -74.0

In [5]:
# Connect to AWS Zarr store
fs = fsspec.filesystem("s3", anon=True)
mapper = fs.get_mapper("noaa-nws-aorc-v1-1-1km/2024.zarr") #update
ds = xr.open_zarr(mapper, consolidated=True)

# Loop over each hour and export NetCDF
for t in times:
    try:
        print(f"Processing {t}...")
        ds_hour = ds.sel(
            time=t,
            latitude=slice(lat_min, lat_max),
            longitude=slice(lon_min, lon_max)
        )
        
        # Build filename
        fname = f"aorc_hourly_{t:%Y%m%d_%Hz}.nc"
        path = rf"E:\HECRAS_project\GMU_v5_backup\08 Data\helene_net\{fname}" #update

        # Save single variable or all
        ds_hour.to_netcdf(path, format="NETCDF4")
        print(f"✅ Saved: {fname}")

    except Exception as e:
        print(f"❌ Failed at {t}: {e}")

Processing 2024-08-01 00:00:00...
✅ Saved: aorc_hourly_20240801_00z.nc
Processing 2024-08-01 01:00:00...
✅ Saved: aorc_hourly_20240801_01z.nc
Processing 2024-08-01 02:00:00...
✅ Saved: aorc_hourly_20240801_02z.nc
Processing 2024-08-01 03:00:00...
✅ Saved: aorc_hourly_20240801_03z.nc
Processing 2024-08-01 04:00:00...
✅ Saved: aorc_hourly_20240801_04z.nc
Processing 2024-08-01 05:00:00...
✅ Saved: aorc_hourly_20240801_05z.nc
Processing 2024-08-01 06:00:00...
✅ Saved: aorc_hourly_20240801_06z.nc
Processing 2024-08-01 07:00:00...
✅ Saved: aorc_hourly_20240801_07z.nc
Processing 2024-08-01 08:00:00...
✅ Saved: aorc_hourly_20240801_08z.nc
Processing 2024-08-01 09:00:00...
✅ Saved: aorc_hourly_20240801_09z.nc
Processing 2024-08-01 10:00:00...
✅ Saved: aorc_hourly_20240801_10z.nc
Processing 2024-08-01 11:00:00...
✅ Saved: aorc_hourly_20240801_11z.nc
Processing 2024-08-01 12:00:00...
✅ Saved: aorc_hourly_20240801_12z.nc
Processing 2024-08-01 13:00:00...
✅ Saved: aorc_hourly_20240801_13z.nc
Proces

# Convert to cumulative do not use

In [36]:
import os
import xarray as xr

In [ ]:
# === SETTINGS ===
input_folder = r"E:\HECRAS_project\GMU_v5_backup\08 Data"
output_folder = os.path.join(input_folder, "AORC_Data1km_Cumulative")
os.makedirs(output_folder, exist_ok=True)

In [38]:
# AORC variable and settings
var_name = 'APCP_surface'     # Confirmed variable name
time_step_hours = 1           # Hourly AORC data



In [ ]:
# Loop through and convert each file
for filename in sorted(os.listdir(input_folder)):
    if filename.endswith(".nc"):
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, filename)

        try:
            # Open the NetCDF file
            ds = xr.open_dataset(input_path)
            if var_name not in ds:
                print(f"⚠️ Variable '{var_name}' not found in {filename}")
                continue

            # Convert rate (e.g., mm/hr) to cumulative (mm)
            precip_rate = ds[var_name]
            precip_cumulative = precip_rate * time_step_hours
            precip_cumulative.attrs['units'] = 'mm'
            precip_cumulative.attrs['long_name'] = 'Hourly Cumulative Precipitation'
            precip_cumulative.attrs['standard_name'] = 'cumulative_precipitation'

            # Replace in dataset and save
            ds[var_name] = precip_cumulative
            ds.to_netcdf(output_path)
            ds.close()

            print(f"✅ Converted: {filename}")

        except Exception as e:
            print(f"❌ Error processing {filename}: {e}")